In [1]:
import json
import requests
import csv
import jsonlines
from flask import Flask, redirect, render_template, request, url_for
import sys
import numpy as np
from collections import defaultdict
import random
import math
import openai
import re
import pandas as pd
import os
#csv.field_size_limit(sys.maxsize)

In [2]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [3]:
openai.api_key = os.environ['openai_api_key']

In [4]:
data = csv.DictReader(open("../IEEE_papers/Raw_data/IEEE VIS papers 1990-2022 - Main dataset.csv"))

In [5]:
dataset = [datum for datum in data]
# dataset = dataset[0:20]

In [7]:
save_json(dataset, r'../IEEE_papers/processed_data/processed_data.json')

In [128]:
def call_gpt(messages, model="gpt-3.5-turbo-0613"):
    completions = openai.ChatCompletion.create(
        model=model,
        n=1,
        stop=None,
        temperature = 0.0,
        messages=messages)
    gpt_response = completions['choices'][0]['message']['content'].strip() 
    return gpt_response   

In [129]:
def merge_sentences(datum_sentences):
    sentence_list = [" ".join(sentence_word_list) for sentence_word_list in datum_sentences] # merge the words into sentences
    paragraph = " ".join(sentence_list)
    return paragraph

In [143]:
def extract_events(sentence):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a state of the art event extraction system. 
                Your task is to extract only the most important event that describes the main research idea from Research paper abstract.
                Strictly extract only one event. This event should be the major research focus of the abstract.
                The events should be human-readable. 
                Reply in JSON format with each line being an event in the format:
                [event];
                The abstract of research papers will be provided by the user.
            """
        },
        { "role": "system", "name": "example_user", "content": "The success of DL can be attributed to hours of parameter and architecture tuning by human experts. Neural Architecture Search (NAS) techniques aim to solve this problem by automating the search procedure for DNN architectures making it possible for non-experts to work with DNNs. Specifically, One-shot NAS techniques have recently gained popularity as they are known to reduce the search time for NAS techniques. One-Shot NAS works by training a large template network through parameter sharing which includes all the candidate NNs. This is followed by applying a procedure to rank its components through evaluating the possible candidate architectures chosen randomly. However, as these search models become increasingly powerful and diverse, they become harder to understand. Consequently, even though the search results work well, it is hard to identify search biases and control the search progression, hence a need for explainability and human-in-the-loop (HIL) One-Shot NAS. To alleviate these problems, we present NAS-Navigator, a visual analytics (VA) system aiming to solve three problems with One-Shot NAS; explainability, HIL design, and performance improvements compared to existing state-of-the-art (SOTA) techniques. NAS-Navigator gives full control of NAS back in the hands of the users while still keeping the perks of automated search, thus assisting non-expert users. Analysts can use their domain knowledge aided by cues from the interface to guide the search. Evaluation results confirm the performance of our improved One-Shot NAS algorithm is comparable to other SOTA techniques. While adding Visual Analytics (VA) using NAS-Navigator shows further improvements in search time and performance. We designed our interface in collaboration with several deep learning researchers and evaluated NAS-Navigator through a control experiment and expert interviews."},
        { "role": "system", "name": "example_system", "content": "[automating the search procedure for DNN architectures using Neural Architecture Search (NAS) techniques];"},         
        { "role": "user", "content": f"This is the research paper abstract:{sentence}"},
    ]
    events = call_gpt(messages)
    return events

In [148]:
def merged(sentence,cl,keywords):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a named entity recognition model. You will be given an abstract of a research paper and the event recognized in that abstract by the user.
                The format of the input defining event in research paper will be:
                [event];
                A list of key words in the abstract will be provided by the user.
                These key words define the main concepts involved in the research paper. 
                Use these key words to find the 4 most important main participants.
                A participant can be a concept, proposed solution, product, algorithm, technique, instruments, tools, processes or phenomena.
                Strictly extract 4 or less main participants.
                format of key words will be :
                "key word 1, key word 2..."  
                Reply in JSON format with each line being an event in the format:
                [event]:[main participant 1],[main participant 2]...;
            """
        },   
        { "role": "system", "name": "example_user", "content": "The success of DL can be attributed to hours of parameter and architecture tuning by human experts. Neural Architecture Search (NAS) techniques aim to solve this problem by automating the search procedure for DNN architectures making it possible for non-experts to work with DNNs. Specifically, One-shot NAS techniques have recently gained popularity as they are known to reduce the search time for NAS techniques. One-Shot NAS works by training a large template network through parameter sharing which includes all the candidate NNs. This is followed by applying a procedure to rank its components through evaluating the possible candidate architectures chosen randomly. However, as these search models become increasingly powerful and diverse, they become harder to understand. Consequently, even though the search results work well, it is hard to identify search biases and control the search progression, hence a need for explainability and human-in-the-loop (HIL) One-Shot NAS. To alleviate these problems, we present NAS-Navigator, a visual analytics (VA) system aiming to solve three problems with One-Shot NAS; explainability, HIL design, and performance improvements compared to existing state-of-the-art (SOTA) techniques. NAS-Navigator gives full control of NAS back in the hands of the users while still keeping the perks of automated search, thus assisting non-expert users. Analysts can use their domain knowledge aided by cues from the interface to guide the search. Evaluation results confirm the performance of our improved One-Shot NAS algorithm is comparable to other SOTA techniques. While adding Visual Analytics (VA) using NAS-Navigator shows further improvements in search time and performance. We designed our interface in collaboration with several deep learning researchers and evaluated NAS-Navigator through a control experiment and expert interviews."},
        { "role": "system", "name": "example_system", "content": "[automating the search procedure for DNN architectures using Neural Architecture Search (NAS) techniques]:[Neural Architecture Search (NAS) techniques],[DNN architectures],[Visual Analytics],[Deep Learning];"},
        { "role": "user", "content": f"This is the list of keywords in the research paper abstract:{keywords}"},     
        { "role": "user", "content": f"This is the input defining the extracted event :{cl}"},  
        { "role": "user", "content": f"This is the research paper abstract:{sentence}"},
    ]
    events = call_gpt(messages)
    return events

In [154]:
def merged_all(sentence,cl):
    messages = [
        { 
            "role": "system", 
            "content": """
                You will be given an abstract of a research paper and the event graph of one event in that article by the user.
                The format of the event graph will be:
                [event]:[main participant 1],[main participant 2];
                Arguments refer to the specific entities, participants, or elements that play specific roles in an event.
                Extract the main arguments and their roles involved in each event.
                Each extracted argument has to play a major role in the event.
                Strictly assign a role to each argument.                
                Reply in JSON format with each line being an event in the format:
                [event]:[main participant 1],[main participant 2]...:[role 1 - argument 1],[role 2 - argument 2],...; 
            """
        },   
        { "role": "system", "name": "example_user", "content": "The success of DL can be attributed to hours of parameter and architecture tuning by human experts. Neural Architecture Search (NAS) techniques aim to solve this problem by automating the search procedure for DNN architectures making it possible for non-experts to work with DNNs. Specifically, One-shot NAS techniques have recently gained popularity as they are known to reduce the search time for NAS techniques. One-Shot NAS works by training a large template network through parameter sharing which includes all the candidate NNs. This is followed by applying a procedure to rank its components through evaluating the possible candidate architectures chosen randomly. However, as these search models become increasingly powerful and diverse, they become harder to understand. Consequently, even though the search results work well, it is hard to identify search biases and control the search progression, hence a need for explainability and human-in-the-loop (HIL) One-Shot NAS. To alleviate these problems, we present NAS-Navigator, a visual analytics (VA) system aiming to solve three problems with One-Shot NAS; explainability, HIL design, and performance improvements compared to existing state-of-the-art (SOTA) techniques. NAS-Navigator gives full control of NAS back in the hands of the users while still keeping the perks of automated search, thus assisting non-expert users. Analysts can use their domain knowledge aided by cues from the interface to guide the search. Evaluation results confirm the performance of our improved One-Shot NAS algorithm is comparable to other SOTA techniques. While adding Visual Analytics (VA) using NAS-Navigator shows further improvements in search time and performance. We designed our interface in collaboration with several deep learning researchers and evaluated NAS-Navigator through a control experiment and expert interviews."},
        { "role": "system", "name": "example_system", "content": "[automating the search procedure for DNN architectures using Neural Architecture Search (NAS) techniques]:[Visual Analytics],[Deep Learning],[Neural Architecture Search (NAS) techniques],[DNN architectures]:[search procedure - Neural Architecture Search (NAS) techniques],[reducing search time - One Shot NAS techniques],[visual analytics system - NAS Navigator];"},   
        { "role": "user", "content": f"This is the input defining the extracted event and the main participants, strictly use only this event for further tasks  :{cl}"},  
        { "role": "user", "content": f"This is the news article:{sentence}"},
    ]
    events = call_gpt(messages)
    return events
                # For each event assign roles that the main participants play.
                # Only assign roles to the main participants given by user input. 
                # These should be the participants heavily involved in the event.

In [150]:
def strip_sentence(sentence):
    if sentence.startswith('The article discussed how'):
        stripped_sentence = sentence.replace('The article discussed how', '').strip()
        stripped_sentence = re.sub(",","",stripped_sentence)
    elif sentence.startswith('The article discussed'):
        stripped_sentence = sentence.replace('The article discussed', '').strip()
        stripped_sentence = re.sub(",","",stripped_sentence)
    else:
        print("!!!")
    return stripped_sentence

In [152]:
dat = json.load(open(r'../IEEE_papers/processed_data/processed_data.json'))
res_events = []
error_datum = []
for index, datum in enumerate(dat):
    print('{}/{}'.format(index, len(dat)))
    sentence = datum['Abstract'].strip()
    keywords = datum['AuthorKeywords'].strip()
    sentence = re.sub("-"," ",sentence)
    cl = extract_events(sentence)
    events = merged(sentence,cl,keywords)
    ev = merged_all(sentence,events)
    datum['events'] = ev
    res_events.append(datum)
save_json(res_events, r'../IEEE_papers/Events/events_merged2.json')

0/20
1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20


In [153]:
from string import punctuation
def post_process_events(dataset):
    for index, datum in enumerate(dataset):
        datum['doc_id'] = index
        datum['events_raw'] = datum['events']
        events_str = datum['events'].split('\n')
        events = []
        for event_str in events_str:
            arguments=[]
            main_characters = []
            event_str = event_str.strip()
            components = event_str.split(':')
            event_type = components[0].strip()
            chars = components[1].split('],[')
            arguments_raw = components[2:]
            for dat in chars:
                dat=re.sub(",","",dat)
                dat = str(dat).replace('[', '').replace(']', '')
                main_characters.append(dat)
            for args in arguments_raw:
                temp = args.split('],[')
                for arg in temp:
                    arg_raw=arg.split(' - ')
                    arg_type=arg_raw[0].strip()
                    args = arg_raw[1:]
                    args_final=' '.join([str(elem) for elem in args]).strip().strip(punctuation)
                    arg_type = str(arg_type).replace('[', '').replace(']', '')
                    arguments.append({arg_type:args_final})
            events.append({'Trigger':event_type, 'Main Participants': main_characters, 'Arguments': arguments})
        datum['events'] = events
    return dataset

dataset = json.load(open(r'../IEEE_papers/Events/events_merged2.json'))
processed_dataset = post_process_events(dataset)
save_json(processed_dataset, r'../IEEE_papers/Result/events_merged2.json')


Frequency 